In [1]:
'''
Determining CoSE from LRS
Intermediate files generated by bedtools
Uses modules installed by conda
'''

'\nDetermining CoSE from files generated by bedtools\nPlease look at project/Knowledgebase/Notes for additional files\nThe only difference from CoSE_1 is that this uses all introns and CoSE_1 uses first intron\n'

In [2]:
# load modules

import os
import re

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import mygene
import scipy

from plotnine import *
import warnings
warnings.filterwarnings('ignore')

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # export pdfs with editable font types in Illustrator

In [4]:
# List of Junctions with overlap of 10 bp on each side. Generated by bedtools
# path to files to be changed by user

Jn_list = ['/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/MUT3.alljns.intron.overlap.plus10.txt',
           '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/MUT2.alljns.intron.overlap.plus10.txt',
           '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/WT1.alljns.intron.overlap.plus10.txt', 
           '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/WT2.alljns.intron.overlap.plus10.txt'
          ]
           
           
    

In [ ]:
# Function to generate Junction overlaps
def make_Jn_overlap(jn):
    namea, nameb = os.path.split(jn)
    namec = nameb.split("_")[0]
    # read junction files
    SJ1=pd.read_csv(jn,
                sep = "\t",
                delimiter = '\t', index_col = False,
                      names =  ['chr_a', 'start_a', 'end_a', 'name_a', 'score_a', 'strand_a','chr_b', 'start_b', 'end_b', 'name_b', 'score_b', 'strand_b',"overlap","span_a","span_b", "diff_span" ]
                   )
    jn["chr_name"] = jn['chr_a']+"_"+jn['start_a'].apply(str)+"_"+jn['end_a'].apply(str)+"_"+jn['strand_a'] # new column chr_name
    SJ1.loc[SJ1['strand_'] == 1, 'strand'] = "+"
    SJ1.loc[SJ1['strand_'] == 2, 'strand'] = "-"
    SJ1['name']=SJ1['chr']+"_"+SJ1['start'].astype(str)+"_"+SJ1['end'].astype(str)+"_"+SJ1['strand'] # new column name 
    SJ1['score']= SJ1['unique']+SJ1['multi_map'] # both single and multimapping reads included
    SJ1=SJ1[SJ1['annotated'] == 1] # only annotated reads included
    SJ1=SJ1.dropna() # those with any NA are dropped
    SJ1=SJ1[['chr', 'start', 'end','name','score','strand']] # reduce file to just standard bed6 format
    folder1='/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_FastGro/Splice_Junctions'
    path1=folder1+'/'+namec+".STAR_Jn.bed" # new folder for all splice junctions
    # write splice junctions to files
    SJ1.to_csv(path1,
            sep = "\t",
             header = None,
             index = False)
    
    print(namea, nameb, namec)
    #print(SJ1.head)
    

In [5]:
# make intron overlap list generated above

intron_list = [
    '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/WT1.all-intron.overlap.txt',
    '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/WT2.all-intron.overlap.txt',
    '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/MUT2.all-intron.overlap.txt',
    '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE/MUT3.all-intron.overlap.txt'
]

In [337]:
# only intron name. Please note difference with the next function intron_overlap2

def intron_overlap(intron):
    namea, nameb = os.path.split(intron)
    namec = nameb.split(".")[0]
    named = nameb.split(".")[1]
    intron1 = pd.read_csv(intron, 
                      delimiter = '\t', 
                          index_col = False,
                      names =  ['chr_a', 'start_a', 'end_a', 'name_a', 'score_a', 'strand_a','chr_b', 'start_b', 'end_b', 'name_b', 'score_b', 'strand_b',"overlap" ])
    
    # passing function
    agg_func_math = {
        'name_a':
        ['count']
            }
    intron_count = intron1.groupby(['name_a']).agg(agg_func_math).round(2)
    intron_count.columns = intron_count.columns.to_flat_index().str.join('_')
    intron_count['name_a'] = intron_count.index
    intron_count.index = np.arange(len(intron_count))
    intron_count_over10 = intron_count[intron_count['name_a_count'] > 10]
    intron_count.columns = ['intron_count', 'name_a']

    
    folder1 = '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/CoSE'
    jn  = folder1+'/'+namec+'.alljns.intron.overlap.plus10.txt'
    jn1 = pd.read_csv(jn,
                delimiter = '\t', 
                      index_col = False,
                      names =  ['chr_a', 'start_a', 'end_a', 'name_a', 'score_a', 'strand_a','chr_b', 'start_b', 'end_b', 'name_b', 'score_b', 'strand_b',"overlap","span_a","span_b", "diff_span" ]
                     )
    jn1["chr_name"] = jn1['chr_a']+"_"+jn1['start_a'].apply(str)+"_"+jn1['end_a'].apply(str)+"_"+jn1['strand_a']
    jn1_count = jn1.groupby(['name_a']).agg(agg_func_math).round(2)
    
    jn1_count.columns = jn1_count.columns.to_flat_index().str.join('_')
    jn1_count['name_a'] = jn1_count.index
    jn1_count.index = np.arange(len(jn1_count))
    

    jn1_count.columns = ['jn_count', 'name_a']
    merged1 = pd.merge(intron_count, jn1_count, on = 'name_a', how = "left")

    merged1 = merged1.fillna(0).round(2)
    merged1['CoSE'] = (merged1['jn_count']/merged1['intron_count']).round(2)
    merged1['sample'] = namec
    merged1 = merged1[merged1["jn_count"]>1]
    #merged1.columns = 
    dest = folder1+'/'+'Final'+'/'+namec+'.'+named+'.CoSE.txt'
    merged1.to_csv(dest, sep = '\t', index = False, header = True) 
    
    print(namea, nameb, namec)
    print(intron_count.head(5))
    #print(intron_count_over10.head(5))
    print(jn1_count.head(10))
    print(merged1.head(5))
    #print(intron_count.columns)
    #print(intron_count_over10.counts)
    print(jn1_count.columns)
    print(merged1.columns)
    print(dest)
    

In [ ]:
for intron in intron_list:
    intron_overlap(intron)